In [1]:
import os

import requests
import zipfile
from pathlib import Path

import numpy 
import matplotlib.pyplot as plt

import spectral as spl

In [2]:
# import torch
# torch.__version__

from torch import nn, optim
import torch.nn.functional as F

import torch  # 导入pytorch
from torch import nn, optim  # 导入神经网络与优化器对应的类
import torch.nn.functional as F 
from torchvision import datasets, transforms ## 导入数据集与数据预处理的方法

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
def plotWithName(theSpecLib, index):
    name = theSpecLib.names[index]
    print(name)
    bandC = theSpecLib.bands.centers
    data = theSpecLib.spectra
    plt.plot(bandC,data[index])

def getIndex(theSpecLib, strFirst, strLast):
    firstIndex = theSpecLib.names.index(strFirst)
    lastIndex = theSpecLib.names.index(strLast)
    return (firstIndex, lastIndex)

In [5]:
SpecLib2287 = spl.envi.open(r'./aster/mineral_jhu_nicolet_2287.hdr')
names = SpecLib2287.names

doloFirstDataIndex = names.index("Dolomite CaMg(CO3)2 [carbonate-none-coarse-dolomi1]")
doloLastDataIndex = names.index("Dolomite CaMg(CO3)2 [carbonate-none-solid-dolomi3]")
oliFirstDataIndex = names.index("Olivine (Fo11) (Fe+2 Mg)2SiO4 [silicate-nesosilicate-fine-olivi10]")
oliLastDataIndex = names.index("Olivine (Fo92) (Fe+2 Mg)2SiO4 [silicate-nesosilicate-solid-olivin13]")

doloIndex = getIndex(SpecLib2287 ,"Dolomite CaMg(CO3)2 [carbonate-none-coarse-dolomi1]", "Dolomite CaMg(CO3)2 [carbonate-none-solid-dolomi3]")
oliIndex = getIndex(SpecLib2287 ,"Olivine (Fo11) (Fe+2 Mg)2SiO4 [silicate-nesosilicate-fine-olivi10]", "Olivine (Fo92) (Fe+2 Mg)2SiO4 [silicate-nesosilicate-fine-olivin1]")

trainData1 = SpecLib2287.spectra[doloIndex[0]:doloIndex[1]] 
trainData2 = SpecLib2287.spectra[oliIndex[0]:oliIndex[1]]
trainData = numpy.vstack((trainData1, trainData2))
# trainTensor = torch.Tensor(trainData)

In [6]:
# Write transform for image
data_transform = transforms.Compose([
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0 
])

In [7]:
import os
import pathlib
import torch

from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from typing import Tuple, Dict, List

In [8]:
def find_classes():
    return (['Oli', 'Dolo'], {'Oli': 0, 'Dolo': 1})

In [9]:
find_classes()[0]

['Oli', 'Dolo']

In [380]:
# 这个需要好好看，有可能会有不少问题！


# Write a custom dataset class (inherits from torch.utils.data.Dataset)
from torch.utils.data import Dataset

# 1. Subclass torch.utils.data.Dataset
class dataLoaderCustom(Dataset):
    
    # 2. Initialize with a targ_dir and transform (optional) parameter
    def __init__(self, targ_dir: str, transform=None) -> None:
        
        # 3. Create class attributes
        # Get all image paths
        self.paths = list(pathlib.Path(targ_dir).glob("*/*.jpg")) 
        # Setup transforms
        self.transform = transform
        # Create classes and class_to_idx attributes
        self.classes, self.class_to_idx = find_classes()
    
#     # 4. Make function to load images
#     def load_image(self, index: int) -> Image.Image:
#         "Opens an image via a path and returns it."
#         image_path = self.paths[index]
#         return Image.open(image_path) 
    
    
    # 4. Make function to load SEPC!!!!!!!!!!!!!
    def load_spec(self, index: int):
#         "Opens an image via a path and returns it."
#         image_path = self.paths[index]
        return trainData[index]
    
    
    # 5. Overwrite the __len__() method (optional but recommended for subclasses of torch.utils.data.Dataset)
    def __len__(self) -> int:
        "Returns the total number of samples."
        return len(trainData)
    
    
    # 6. Overwrite the __getitem__() method (required for subclasses of torch.utils.data.Dataset)
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        
        "Returns one sample of data, data and label (X, y)."
        
        spec = self.load_spec(index)
        
        if index < 10:
            class_name, class_idx = "Oli", 0
        else:
            class_name, class_idx = "Dolo", 1
        
#         class_name  = self.paths[index].parent.name # expects path in data_folder/class_name/image.jpeg
        
#         class_idx = self.class_to_idx[class_name]

        # Transform if necessary
        if self.transform:
            return self.transform(spec), class_idx # return data, label (X, y)
        else:
            return spec, class_idx # return data, label (X, y)
    
    
#     def check():
#         print(self.class)
#         print(self.class_idx)
    
    
    

In [381]:
train_data_custom = dataLoaderCustom(targ_dir='train_dir', transform=torch.Tensor)


# train_data_custom.load_spec(12), train_data_custom.__len__(), train_data_custom.__getitem__(19)


In [383]:
train_dataloader_custom = DataLoader(dataset=train_data_custom, # use custom created train Dataset
                                     batch_size=1, # how many samples per batch?
                                     num_workers=0, # how many subprocesses to use for data loading? (higher = more)
                                     shuffle=True) # shuffle the data?
train_dataloader_custom

In [384]:
spec_custom, label_custom = next(iter(train_dataloader_custom))

print(f"Image shape: {spec_custom.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label_custom.shape}")

Image shape: torch.Size([1, 2287]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([1])


In [385]:
# 模型定义
class isOliClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        
#         modelVan = nn.Linear(2287, 4) # 输入特征数为2，输出特征数为1
        
        self.fc1 = nn.Linear(2287, 64)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 2)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
    
#         print(self.fc1)
    
        
        return x

In [386]:
### 这个被我过分简化了,现在重写!


# 对上面定义的Classifier类进行实例化
model = isOliClassifier()

# 定义损失函数为负对数损失函数
criterion = nn.NLLLoss()

# 优化方法为Adam梯度下降方法，学习率为0.003
optimizer = optim.Adam(model.parameters(), lr=0.003)

# 对训练集的全部数据学习15遍，这个数字越大，训练时间越长
# epochs = 31

# 将每次训练的训练误差和测试误差存储在这两个列表里，后面绘制误差变化折线图用
train_losses, test_losses = [], []

print('开始训练')


# for i in trainTensor:

for spec, labels in train_dataloader_custom:
    
    running_loss = 0

    optimizer.zero_grad()

    # 对64张图片进行推断，计算损失函数，反向传播优化权重，将损失求和

    log_ps = model(spec.view(1,2287))
    loss = criterion(log_ps, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    
    print(running_loss)


开始训练
0.0008581769652664661
175.85501098632812
12.494743347167969
15.063606262207031
31.296154022216797
1.311301275563892e-06
15.022822380065918
16.616193771362305
6.057568073272705
5.827556610107422
0.34787994623184204
0.0026280886959284544
4.887569048150908e-06
13.998854637145996
20.005277633666992
0.030921922996640205
0.22494037449359894
5.021701335906982
0.0
7.073669910430908
6.214914321899414
1.919238567352295
0.34241974353790283
0.006333636119961739
0.004869387950748205
9.13963794708252
12.376642227172852
0.0007104733376763761
10.250435829162598
0.0001134808044298552
0.005305496044456959
0.03254403546452522


In [388]:
# 对上面定义的Classifier类进行实例化
model = isOliClassifier()

# 定义损失函数为负对数损失函数
criterion = nn.NLLLoss()

# 优化方法为Adam梯度下降方法，学习率为0.003
optimizer = optim.Adam(model.parameters(), lr=0.003)

# 对训练集的全部数据学习15遍，这个数字越大，训练时间越长
epochs = 100

# 将每次训练的训练误差和测试误差存储在这两个列表里，后面绘制误差变化折线图用
train_losses, test_losses = [], []

print('开始训练')
for e in range(epochs):
    running_loss = 0
    
    # 对训练集中的所有图片都过一遍
    for spec, labels in train_dataloader_custom:
        # 将优化器中的求导结果都设为0，否则会在每次反向传播之后叠加之前的
        optimizer.zero_grad()
        
        # 对64张图片进行推断，计算损失函数，反向传播优化权重，将损失求和
        log_ps = model(spec)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # 每次学完一遍数据集，都进行以下测试操作
    else:
        test_loss = 0
        accuracy = 0
        # 测试的时候不需要开自动求导和反向传播
        with torch.no_grad():
            # 关闭Dropout
            model.eval()
            
            # 对测试集中的所有图片都过一遍
            for spec, labels in train_dataloader_custom:
                # 对传入的测试集图片进行正向推断、计算损失，accuracy为测试集一万张图片中模型预测正确率
                log_ps = model(spec)
                test_loss += criterion(log_ps, labels)
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                
                # 等号右边为每一批64张测试图片中预测正确的占比
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        # 恢复Dropout
        model.train()
        # 将训练误差和测试误差存在两个列表里，后面绘制误差变化折线图用
        train_losses.append(running_loss/len(train_dataloader_custom))
        test_losses.append(test_loss/len(train_dataloader_custom))

        print("训练集学习次数: {}/{}.. ".format(e+1, epochs),
              "训练误差: {:.3f}.. ".format(running_loss/len(train_dataloader_custom)),
              "测试误差: {:.3f}.. ".format(test_loss/len(train_dataloader_custom)),
              "模型分类准确率: {:.3f}".format(accuracy/len(train_dataloader_custom)))

开始训练
训练集学习次数: 1/100..  训练误差: 47.049..  测试误差: 0.655..  模型分类准确率: 0.688
训练集学习次数: 2/100..  训练误差: 0.654..  测试误差: 0.652..  模型分类准确率: 0.688
训练集学习次数: 3/100..  训练误差: 0.649..  测试误差: 0.647..  模型分类准确率: 0.688
训练集学习次数: 4/100..  训练误差: 0.645..  测试误差: 0.642..  模型分类准确率: 0.688
训练集学习次数: 5/100..  训练误差: 0.641..  测试误差: 0.639..  模型分类准确率: 0.688
训练集学习次数: 6/100..  训练误差: 0.638..  测试误差: 0.637..  模型分类准确率: 0.688
训练集学习次数: 7/100..  训练误差: 0.636..  测试误差: 0.634..  模型分类准确率: 0.688
训练集学习次数: 8/100..  训练误差: 0.633..  测试误差: 0.632..  模型分类准确率: 0.688
训练集学习次数: 9/100..  训练误差: 0.632..  测试误差: 0.630..  模型分类准确率: 0.688
训练集学习次数: 10/100..  训练误差: 0.630..  测试误差: 0.629..  模型分类准确率: 0.688
训练集学习次数: 11/100..  训练误差: 0.629..  测试误差: 0.627..  模型分类准确率: 0.688
训练集学习次数: 12/100..  训练误差: 0.627..  测试误差: 0.626..  模型分类准确率: 0.688
训练集学习次数: 13/100..  训练误差: 0.626..  测试误差: 0.626..  模型分类准确率: 0.688
训练集学习次数: 14/100..  训练误差: 0.626..  测试误差: 0.624..  模型分类准确率: 0.688
训练集学习次数: 15/100..  训练误差: 0.625..  测试误差: 0.624..  模型分类准确率: 0.688
训练集学习次数: 16/100..  训练误差: 0.625..  测试误差: 0.6

In [376]:
25/(6+25)

0.8064516129032258

In [379]:
index = 27

if index < 10:
    class_name, class_idx = "Oli", 0
else:
    class_name, class_idx = "Dolo", 1

In [128]:
class_name, class_idx

('Dolo', 1)